#### Import


In [21]:
import os 
import json
import requests
import datetime
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator


#### Utilis


In [ ]:
def translate(source='pl', target = 'en'):
        return  GoogleTranslator(source,target).translate(text)

In [29]:
def extract(ancestor, selector=None, attribute=None, multiple=False):
    if selector:
        if multiple: 
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [p.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one (selector)[attribute].strip()
            except TypeError:
                return None 
        try:
            return ancestor.select_one (selector).text.strip()
        except AttributeError:
            return None 
    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()

In [38]:
selectors = {
'opinion_id': (None,"data-entry-id"),
'author':("span.user-post__author-name",),
'recommend' :("span.user-post__author-recomendation>em",),
'stars': ( "span.user-post__score-count",),
'content_pl': ( "div.user-post__text",),
'pros_pl':("div.review-feature__item--negative", None, True),
'cons_pl': ("div.review-feature__item--negative", None, True),
'up_votes':("button.vote-yes","data-total-vote" ),
'down_votes':("button.vote-no", "data-total-vote"),
'published':("span.user-post__published > time:nth-child(1)","datetime"),
'published':("span.user-post__published > time:nth-child(2)","datetime")
}

#### Extraction of opiniopns

In [33]:
with open("./cookie.json", "r") as jf:
    headers = json.load(jf)

#### URL code


In [56]:
product_id = "84514582"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while url:
    response = requests.get(url, headers = headers )
    if response.status_code == 200:
        print(url)
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(opinions))
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *values)
                for key, values in selectors.items()
            }
            single_opinion['content_en']= translate(single_opinion['content_pl'])
            single_opinion['pros_en']= [translate(pros) for pros in single_opinion['pros_pl']]
            single_opinion['cons_en']= [translate(cons) for cons in single_opinion['cons_pl']]
            single_opinion['recommend']= True if single_opinion['recommend'] == "Polecam" else False if single_opinion['recommend'] == "Nie polecam" else None 
            single_opinion['stars']= float(single_opinion['stars'].split("/")[0].replace(",", "."))
            # single_opinion['published'] = datetime.datetime.formisoformat(single_opinion['published'])
            # single_opinion['purchased'] = datetime.datetime.formisoformat(single_opinion['published'])
            single_opinion['up_votes'] = int(single_opinion['up_votes'])
            single_opinion['down_votes'] = int(single_opinion['down_votes'])
            all_opinions.append(single_opinion)
        try:
            url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None


https://www.ceneo.pl/84514582#tab=reviews
10


NameError: name 'p' is not defined

#### Creation of database 

In [55]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [48]:
with open (f"./opinions/{product_id}.json", "w", encoding = "UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)